In [67]:
import pandas as pd
import re
import gensim
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import *
import numpy as np
np.random.seed(2018)
import nltk
import nltk.stem as stemmer
stemmer = PorterStemmer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [68]:
File = pd.ExcelFile('DatosWoS2020.xlsx')
df=File.parse('Sheet1')

In [381]:
docktype = pd.read_csv('Docktypes.csv')
docktype = docktype["tipo"]

In [384]:
docktype.describe()

,tipo
count,621
unique,13
top,Article
freq,427


In [70]:
titles = df["Titulo"]

In [71]:
titles_list = titles.values.tolist()


In [72]:
for i in range(len(titles_list)):
    titles_list[i] = str(titles_list[i])
    titles_list[i] = re.sub('[!,*)@#%(&$_?.^:-]', '', titles_list[i])
    titles_list[i] = titles_list[i].replace('"','')
    titles_list[i] = titles_list[i].lower()
    titles_list[i] = titles_list[i].replace("movie","")


In [73]:
indexs = []
for i in range(len(titles_list)):
    indexs.append(i)

In [74]:
df2 = pd.DataFrame(list(zip(titles_list, indexs)),
               columns =['Titulo', 'index'])

In [383]:
df2.head()

,index
count,1790.000000
mean,894.500000
std,516.872808
min,0.000000
25%,447.250000
50%,894.500000
75%,1341.750000
max,1789.000000


In [76]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [77]:
documents = df2
doc_sample = documents[documents['index'] == 0].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', 'industry', 'demand', 'and', 'theater', 'availability']


 tokenized and lemmatized document: 
['industri', 'demand', 'theater', 'avail']


In [78]:
processed_docs = documents['Titulo'].map(preprocess)
processed_docs[:10]

0                   [industri, demand, theater, avail]
1                                   [success, predict]
2                         [sentiment, analysi, review]
3                                      [varieti, citi]
4      [multimod, approach, multilabel, genr, classif]
5         [camo, contextawar, ontolog, gener, databas]
6                [exploit, user, review, automat, tag]
7    [squar, support, vector, regress, forecast, sa...
8                        [recommend, system, interpol]
9    [popular, predict, statist, model, machin, lea...
Name: Titulo, dtype: object

In [79]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 avail
1 demand
2 industri
3 theater
4 predict
5 success
6 analysi
7 review
8 sentiment
9 citi
10 varieti


In [212]:
a = len(processed_docs)
b = len(dictionary)+1
print(a)
print(b)

1790
4320


In [99]:
newindexs = []
for i in range(len(newtitles_list)):
    newindexs.append(i)

In [100]:
newdf2 = pd.DataFrame(list(zip(newtitles_list, newindexs)),
               columns =['Titulo', 'index'])

In [81]:
newtitles_list = titles.values.tolist()
for i in range(len(newtitles_list)):
    newtitles_list[i] = str(newtitles_list[i])
    newtitles_list[i] = re.sub('[!,*)@#%(&$_?.^:-]', '', newtitles_list[i])
    newtitles_list[i] = newtitles_list[i].replace('"','')
    newtitles_list[i] = newtitles_list[i].lower()

In [104]:
newdocuments = newdf2
newdoc_sample = newdocuments[newdocuments['index'] == 0].values[0][0]
print('original document: ')
newwords = []
for newword in newdoc_sample.split(' '):
    newwords.append(newword)
print(newwords)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(newdoc_sample))

original document: 
['movie', 'industry', 'demand', 'and', 'theater', 'availability']


 tokenized and lemmatized document: 
['movi', 'industri', 'demand', 'theater', 'avail']


In [110]:
newlist_titles_processed = []
newprocessed_docs = documents['Titulo'].map(preprocess)
newprocessed_docs[:10]
for i in range(len(newprocessed_docs)):
    newlist_titles_processed.append(' '.join(newprocessed_docs[i]))

In [429]:
datarray = []
for i in range(a):
    datarray_slice = []
    for j in range(b):
        if  j == 0:
            datarray_slice.append(newtitles_list[i])
        else:
            datarray_slice.append("0")
    datarray.append(datarray_slice)

In [430]:
for i in range(a):
    for j in range(1,b):
        for k in range(len(newprocessed_docs[i])):
            if dictionary[j-1]  == newprocessed_docs[i][k]:
                datarray[i][j] = "1"

In [431]:
frcount = []
for i in range(1,len(dictionary)+1):
    count = 0
    for j in range(len(datarray)):
        count = count + int(datarray[j][i])
    namecount = [dictionary[i-1],count]
    frcount.append(namecount)

In [432]:
for i in range(len(datarray)):
    count += int(datarray[i][1])
print(count)

3


In [433]:
print(dictionary[0])

avail


In [434]:
sorted_frcount = sorted(frcount, key=lambda x: x[1], reverse=True)
quince_mayores = []
for i in range(15):
    quince_mayores.append(sorted_frcount[i])
print(quince_mayores)

[['recommend', 127], ['analysi', 112], ['film', 106], ['base', 95], ['learn', 86], ['studi', 84], ['cinema', 79], ['model', 78], ['video', 64], ['imag', 64], ['network', 63], ['social', 57], ['effect', 55], ['sentiment', 54], ['approach', 53]]


In [435]:
coincidence_list = []
procecced_coincidece_list = []
for i in range(len(newtitles_list)):
    for j in range(len(quince_mayores)):
        for k in range(len(newprocessed_docs[i])):
            if quince_mayores[j][0] == newprocessed_docs[i][k]:
                print("la frase " + quince_mayores[j][0] + " aparece en " + titles_list[i])
                if (titles_list[i] in coincidence_list) == False:
                    coincidence_list.append(titles_list[i])
                    procecced_coincidece_list.append(newprocessed_docs[i])
                    break

la frase analysi aparece en sentiment analysis on  reviews
la frase sentiment aparece en sentiment analysis on  reviews
la frase approach aparece en a multimodal approach for multilabel  genre classification
la frase recommend aparece en  recommender systems made through tag interpolation
la frase learn aparece en popularity prediction of s from statistical modeling to machine learning techniques
la frase model aparece en popularity prediction of s from statistical modeling to machine learning techniques
la frase film aparece en the speech genre of the  announcement on the material of  announcements of indian films
la frase analysi aparece en the effects of  piracy on boxoffice revenue an empirical analysis of the chinese  market
la frase effect aparece en the effects of  piracy on boxoffice revenue an empirical analysis of the chinese  market
la frase learn aparece en how words matter machine learning   success
la frase learn aparece en  rating prediction using ensemble learning algor

la frase studi aparece en a comparative study of depiction of feminism on indian celluloid in pre and postliberalisation era tribute to a visionary maestro
la frase network aparece en paratexts on a social network site and their relevance in the production of meaningresults of a qualitative investigation of twitterfeeds
la frase social aparece en paratexts on a social network site and their relevance in the production of meaningresults of a qualitative investigation of twitterfeeds
la frase learn aparece en even a twoyearold can do it the early stages of learning to understand movingimage media
la frase recommend aparece en coldstart recommendation for ondemand cinemas
la frase cinema aparece en coldstart recommendation for ondemand cinemas
la frase approach aparece en modernity criticism and return of religion an approach of parasite and the platform
la frase studi aparece en an analytical study of the ritual ceremonies in iranian performing arts a case study of travellers
la frase re

la frase studi aparece en a multicomponent intervention to decrease sedentary time during hospitalization a quasiexperimental pilot study
la frase film aparece en emergence of indian film industry in the international markets facilitators and impeders
la frase film aparece en film in an ipe classroom for a critical pedagogy of the everyday
la frase film aparece en a georgia civil war film with a bosnian war subtext the case of john frankenheimer's andersonville
la frase analysi aparece en extracting a human feeling from a text a natural language processing task called sentiment analysis using a recurrent neural network together with keras library
la frase network aparece en extracting a human feeling from a text a natural language processing task called sentiment analysis using a recurrent neural network together with keras library
la frase sentiment aparece en extracting a human feeling from a text a natural language processing task called sentiment analysis using a recurrent neural n

la frase network aparece en research on network public opinion analysis and monitor method based on big data technology
la frase network aparece en hybrid tiled convolutional neural networks htcnn text sentiment classification
la frase sentiment aparece en hybrid tiled convolutional neural networks htcnn text sentiment classification
la frase studi aparece en the teachings of the epidemic thrillers to the study of the covid19
la frase learn aparece en design of desktop audiovisual entertainment system with deep learning and haptic sensations
la frase base aparece en computervision based secondorder kineticcolor data generation arsenic quantitation in natural waters
la frase analysi aparece en not an ounce of hollywood bullshit a narrative analysis of news media coverage of spotlight's oscar win
la frase learn aparece en detecting gender stereotypes lexicon vs supervised learning methods
la frase film aparece en the cold war and the sea politics of memory and the submarine film
la frase

la frase cinema aparece en rainforest sublime in cinema a postanthropocentric amazonian aesthetics
la frase recommend aparece en tutorial on conversational recommendation systems
la frase analysi aparece en prediction of factors of pincer nail using gravity center fluctuation based on factor analysis
la frase base aparece en prediction of factors of pincer nail using gravity center fluctuation based on factor analysis
la frase approach aparece en a contentbased approach to profile expansion
la frase video aparece en enhancement of noisy and compressed videos by optical flow and nonlocal denoising
la frase recommend aparece en j3r joint multitask learning of ratings and review summaries for explainable recommendation
la frase learn aparece en j3r joint multitask learning of ratings and review summaries for explainable recommendation
la frase recommend aparece en explainable recommendations via attentive multipersona collaborative filtering
la frase learn aparece en hierarchical reinforc

la frase base aparece en oslcfit organic simultaneous lstm and cnn fit a novel deep learning based solution for sentiment polarity classification of reviews
la frase learn aparece en oslcfit organic simultaneous lstm and cnn fit a novel deep learning based solution for sentiment polarity classification of reviews
la frase sentiment aparece en oslcfit organic simultaneous lstm and cnn fit a novel deep learning based solution for sentiment polarity classification of reviews
la frase model aparece en modeling an auditory stimulated brain under altered states of consciousness using the generalized ising model
la frase approach aparece en intersubject correlations during natural viewing a filterbank approach
la frase recommend aparece en itembased collaborative filtering and association rules for a baseline recommender in ecommerce
la frase imag aparece en the damaging effects of perceived crocodile tears for a crier's image
la frase effect aparece en the damaging effects of perceived croco

la frase cinema aparece en research into cinema fostering tourism a methodological review
la frase effect aparece en the effect of piracy website blocking on consumer behavior
la frase video aparece en playing with feelings video games and affect
la frase film aparece en how motion shapes ought in cinema the embodied film style of eric rohmer
la frase cinema aparece en how motion shapes ought in cinema the embodied film style of eric rohmer
la frase video aparece en video games and the mind essays on cognition affect and emotion
la frase effect aparece en effects of exposure to multiple graphic suicide news articles on explicit and implicit measures of suicide risk
la frase film aparece en stanley cavell and film scepticism and selfreliance at the cinema
la frase cinema aparece en stanley cavell and film scepticism and selfreliance at the cinema
la frase learn aparece en analytic approaches and critical practices on what we can learn
la frase approach aparece en analytic approaches and

In [436]:
datacoincidence = []
for i in range(len(coincidence_list)):
    datacoincidence_slice = []
    for j in range(len(quince_mayores)+1):
        if  j == 0:
            datacoincidence_slice.append(newtitles_list[i])
        else:
            datacoincidence_slice.append("0")
    datacoincidence.append(datacoincidence_slice)

In [437]:
for i in range(len(coincidence_list)):
    for j in range(1,len(quince_mayores)+1):

        for k in range(len(procecced_coincidece_list[i])):
            if quince_mayores[j-1][0]  == procecced_coincidece_list[i][k]:
                datacoincidence[i][j] = "1"

In [438]:
print(datacoincidence[0])

['movie industry demand and theater availability', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0']


In [439]:
citations = df["Citas"]

In [440]:
references = df["referencias"]

In [457]:
for i in range(len(coincidence_list)):
    for j in range(len(titles_list)):
        if coincidence_list[i] == titles_list[j]:
            datacoincidence[i].append(citations[j])
            datacoincidence[i].append(references[j])
            datacoincidence[i].append(docktype[j])

In [461]:
print(datacoincidence[823])

['diversity of olfactory responses and skills in astyanax mexicanus cavefish populations inhabiting different caves', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '13', 53, 'articulo']


In [462]:
import xlsxwriter
encabezados = []
doctypes = docktype.values.tolist()
for i in range(len(quince_mayores)):
    encabezados.append(quince_mayores[i][0])
encabezados.insert(0,"titulos")
encabezados.append("citas")
encabezados.append("referencias")
encabezados.append("clase")
encabezados = [encabezados]

datacoincidence = encabezados + datacoincidence
workbook = xlsxwriter.Workbook('quincepalabras.xlsx')
worksheet = workbook.add_worksheet()
array = datacoincidence
row = 0

for col, data in enumerate(array):
    worksheet.write_row(col, row, data)

workbook.close()

In [449]:
print(len(datacoincidence))

824


In [455]:
docktype[823]

'articulo'

In [446]:

docktype.drop(index=docktype.index[-1], 
        axis=0, 
        inplace=True)

In [447]:
docktype.describe()

count         824
unique         14
top       Article
freq          427
Name: tipo, dtype: object

In [456]:
citations.describe()

count     1790
unique      44
top          -
freq       824
Name: Citas, dtype: object